In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124

In [ ]:
import requests
from transformers import AutoTokenizer, AutoModel
from qalsadi.lemmatizer import Lemmatizer
import torch
import torch.nn.functional as F

# ====== التحميل ======
tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v3")
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3")
lemmatizer = Lemmatizer()

# تشغيل على CPU لتجنب مشاكل Meta tensor
device = "cpu"
model.to(device)

# ====== تحميل البيانات ======
request = requests.get('https://naama.sa/services/getservicesearch')
data = request.json()
print("عدد العناصر في الداتا:", len(data))

# ====== Lemmatization ======
def lemmatize_text(text):
    return " ".join(lemmatizer.lemmatize(word) for word in text.split())

# ====== دالة إمبد batch ======
def embed_batch(texts, batch_size=512):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        # نقل البيانات للـ CPU + تحويل float32
        inputs = {k: v.to(device, dtype=torch.float32) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.float().mean(dim=1)
            embeddings_list.append(batch_embeddings)
    embeddings = torch.cat(embeddings_list, dim=0)
    return embeddings

# ====== عمل إمبد لكل البيانات مرة واحدة ======
all_texts = [item['value'] for item in data]
print("Embedding all data... قد ياخذ وقت حسب حجم الداتا")
data_embeddings = embed_batch(all_texts)
print("تم الانتهاء من إمبد الداتا!")

# ====== البحث بالكويري فقط ======
def search_query(query, data, data_embeddings, top_k=5):
    # إمبد الكويري فقط
    query_embedding = embed_batch([query])
    # حساب التشابه باستخدام cosine similarity
    similarities = F.cosine_similarity(query_embedding, data_embeddings)
    # ترتيب النتائج حسب التشابه
    top_indices = similarities.topk(top_k).indices.tolist()
    results = [data[i] for i in top_indices]
    return results

# ====== مثال بحث ======
query = "قطوه"
results = search_query(query, data, data_embeddings, top_k=5)

print("\nResults:", len(results))
for r in results:
    print("-", r['value'])


ImportError: cannot import name 'is_contiguous_for_memory_format_or_false' from 'torch._prims_common' (/usr/local/lib/python3.12/dist-packages/torch/_prims_common/__init__.py)